<a href="https://colab.research.google.com/github/Chockalingam-Palaniappan/Major-Project/blob/main/Sentiment_analysis_ML_major_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 5.4MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [4]:
urls = ['https://inshorts.com/en/read/sports',
        'https://inshorts.com/en/read/technology',
        'https://inshorts.com/en/read/entertainment']

def build_dataset(urls):
  news_data = []
  for url in urls:
    news_category  = url.split('/')[-1]
    data = requests.get(url)
    soup = BeautifulSoup(data.content)

    news_articles = [{'news_headline':headline.find('span',attrs={"itemprop":"headline"}).string,
                      'news_article':article.find('div',attrs={"itemprop":"articleBody"}).string,
                      'news_category':news_category}
                     
                     for headline,article in zip(soup.find_all('div', class_=["news-card-title news-right-box"]),
                                                 soup.find_all('div', class_=["news-card-content news-right-box"]))]

    news_articles = news_articles[0:30]
    news_data.extend(news_articles)

  df = pd.DataFrame(news_data)
  df = df[['news_headline','news_article','news_category']]
  return df

In [5]:
df = build_dataset(urls)
df.head()

,news_headline,news_article,news_category
0,Rajasthan Royals gift jersey to Stokes with hi...,Ruled out of the remainder of IPL 2021 after f...,sports
1,India's Jhilli Dalabehera bags gold at Asian W...,India's Jhilli Dalabehera won gold in the 45 k...,sports
2,Athiya Shetty shares pic with KL Rahul on his ...,Wishing cricketer KL Rahul on the occasion of ...,sports
3,Dhanashree's reaction to Chahal's 1st wicket i...,Yuzvendra Chahal's wife Dhanashree Verma's rea...,sports
4,Tackling dew is one of the major factors: Anri...,"Ahead of their game against Punjab Kings, Delh...",sports


In [6]:
df.to_csv('news.cv',index=False)

In [7]:
import nltk
nltk.download('stopwords')
stopwords_list = nltk.corpus.stopwords.words('english')
stopwords_list.remove('no')
stopwords_list.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def html_tag(text):
  soup = BeautifulSoup(text,"html.parser")
  new_text = soup.get_text()
  return new_text

In [9]:
!pip install contractions
import contractions
def con(text):
  expand = contractions.fix(text)
  return expand

     |████████████████████████████████| 327kB 5.4MB/s 
     |████████████████████████████████| 286kB 7.3MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85392 sha256=d432a7a479a19be3b12fb2ea08e88fb6b11c216f4511ac9486970870b3213176
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [10]:
import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text = re.sub(pattern,'',text)
  return text

In [11]:
from nltk.tokenize.toktok import  ToktokTokenizer
tokenizer = ToktokTokenizer()

In [12]:
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopwords_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

In [13]:
df.news_headline = df.news_headline.apply(lambda x:x.lower())
df.news_article = df.news_article.apply(lambda x:x.lower())

df.news_headline = df.news_headline.apply(html_tag)
df.news_article = df.news_article.apply(html_tag)

df.news_headline = df.news_headline.apply(con)
df.news_article = df.news_article.apply(con)

df.news_headline = df.news_headline.apply(remove_sp)
df.news_article = df.news_article.apply(remove_sp)

df.news_headline = df.news_headline.apply(remove_stopwords)
df.news_article = df.news_article.apply(remove_stopwords)

In [14]:
df['compound'] = df['news_article'].apply(lambda x: vs.polarity_scores(x)['compound'])
df

,news_headline,news_article,news_category,compound
0,rajasthan royals gift jersey stokes late fathe...,ruled remainder ipl 2021 fracturing left index...,sports,-0.9169
1,indias jhilli dalabehera bags gold asian weigh...,indias jhilli dalabehera gold 45 kg category a...,sports,0.7430
2,athiya shetty shares pic kl rahul birthday say...,wishing cricketer kl rahul occasion 29th birth...,sports,0.9169
3,dhanashrees reaction chahals 1st wicket ipl 20...,yuzvendra chahals wife dhanashree vermas react...,sports,0.6433
4,tackling dew one major factors anrich nortje,ahead game punjab kings delhi capitals pacer a...,sports,0.7579
...,...,...,...,...
69,big brother super proud saba saif ali khan,saba ali khan took instagram account saturday ...,entertainment,0.9274
70,hurt rest life paris hilton 2004 sex tape leak,opening leaked 2004 sex tape american socialit...,entertainment,-0.7003
71,athiya shetty shares pic kl rahul birthday say...,wishing cricketer kl rahul occasion 29th birth...,entertainment,0.9169
72,equal pay male costars biggest struggle gal gadot,wonder woman actress gal gadot spoke fighting ...,entertainment,-0.8020


In [15]:
import joblib
joblib.dump(df,'class')

['class']

In [16]:
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

     |████████████████████████████████| 8.2MB 5.3MB/s 
     |████████████████████████████████| 4.2MB 48.9MB/s 
     |████████████████████████████████| 112kB 53.5MB/s 
     |████████████████████████████████| 163kB 54.7MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 122kB 59.3MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=010b942a56211cbf8355b2008d95f9abcb733e45593d1cb4fd5831f6375ceb06
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=917b87f762a79123b37345b9f214dc4778171b8efa8b68fe81d6cdaeb9f72397
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [19]:
%%writefile app.py
import streamlit as st
import joblib
model = joblib.load('class')
st.title('Sentiment Analyser')
print(df)

Overwriting app.py


In [18]:
!nohup streamlit run app.py &
url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://bd1148218ade.ngrok.io'